<a href="https://colab.research.google.com/github/Abdellahbado/Flower-Pictures-Classifier-Using-Transfer-Learning-/blob/main/flower_image_classifier_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow_datasets as tfds

In [4]:
import tensorflow as tf
from tensorflow import keras

In [5]:
dataset, info = tfds.load("tf_flowers", with_info=True, as_supervised=True)
dataset_size = info.splits["train"].num_examples

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [6]:
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

In [7]:
test_split, valid_split, train_split = [
    "train[:10%]",
    "train[10:25%]",
    "train[25:]",
]
test_set = tfds.load("tf_flowers", split=test_split, as_supervised=True)
valid_set = tfds.load("tf_flowers", split=valid_split, as_supervised=True)
train_set = tfds.load("tf_flowers", split=train_split, as_supervised=True)

### Preprocess the image to resize them


In [8]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [9]:
batch_size = 32
train_set = train_set.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)

In [10]:
base_model = keras.applications.xception.Xception(weights="imagenet",
include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

83683744/83683744 [==============================] - 1s 0us/step


In [14]:
for layer in base_model.layers:
  layer.trainable = False

In [12]:
optimizer = keras.optimizers.legacy.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(train_set, epochs=5, validation_data=valid_set)

Epoch 1/5
114/114 [==============================] - 82s 558ms/step - loss: 0.9079 - accuracy: 0.6856 - val_loss: 387.8083 - val_accuracy: 0.2341
Epoch 2/5
114/114 [==============================] - 60s 524ms/step - loss: 0.4832 - accuracy: 0.8335 - val_loss: 2.4238 - val_accuracy: 0.7060
Epoch 3/5
114/114 [==============================] - 59s 520ms/step - loss: 0.2522 - accuracy: 0.9114 - val_loss: 0.1805 - val_accuracy: 0.9474
Epoch 4/5
114/114 [==============================] - 59s 521ms/step - loss: 0.1520 - accuracy: 0.9462 - val_loss: 0.1183 - val_accuracy: 0.9655
Epoch 5/5
114/114 [==============================] - 60s 529ms/step - loss: 0.0846 - accuracy: 0.9706 - val_loss: 0.1029 - val_accuracy: 0.9637


In [ ]:
for layer in base_model.layers:
    layer.trainable = True

In [13]:
optimizer = keras.optimizers.legacy.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(train_set, epochs=5, validation_data=valid_set)

Epoch 1/5
114/114 [==============================] - 65s 531ms/step - loss: 0.3635 - accuracy: 0.8801 - val_loss: 0.4789 - val_accuracy: 0.8639
Epoch 2/5
114/114 [==============================] - 59s 516ms/step - loss: 0.1730 - accuracy: 0.9427 - val_loss: 0.1200 - val_accuracy: 0.9583
Epoch 3/5
114/114 [==============================] - 60s 529ms/step - loss: 0.0780 - accuracy: 0.9753 - val_loss: 0.0382 - val_accuracy: 0.9927
Epoch 4/5
114/114 [==============================] - 60s 527ms/step - loss: 0.0384 - accuracy: 0.9866 - val_loss: 0.0192 - val_accuracy: 0.9946
Epoch 5/5
114/114 [==============================] - 60s 527ms/step - loss: 0.0175 - accuracy: 0.9953 - val_loss: 0.0028 - val_accuracy: 1.0000
